# Udacity -- Self Driving Car Nano Degree
# Behavioral Cloning

### John Mansell
---

### Included Files
> [model.py](model.py) -- The script to create and train the model  
> [drive.py](drive.py) -- Drives the car in autonomous mode  
> [model.h5](model.h5) -- The trained convolutional Neural Network  
> [Write up](WriteUp.ipynb) -- Write Up for the project

# Behavioral Cloning
> This project focusses on behavioral cloning. The idea is to expose the computer to a variety of driving conditions as well as the "correct action" for that particular situation. 

> For example, in our project, a human drives around a track. A camera at the front of the car records the track and the current steering angle. Then, all that data is fed to a neural network. The computer then searches for patterns. The goal is to produce a model where the computer can predict the correct steering angle given an input image. If it can correctly predict the steering angle, then the computer can safely drive the car arround the track.

> This approach is known as behavioral cloning because we don't program rules into the algorithm. Rather, the computer analyzes human behavior and tries to replicate it.

# Gathering the Data
> The first step was to gather the data. Using the simulator provided by Udacity, I drove the car in "Training Mode" and recorded data. The simulator saves images taken from a simulated camera in the front of the car and records the steering angle. Below is an example of the recoded images.
![](writeup_images/road_1_angle.png)

# Processing the Images
> Before passing the images to the nerual network for training, it helps to preprocess the image. The goal is to create a pipeline which will isolate the important information so that the neural network will be trained on the relevant information, not on extranious data.

> ### Image Mask
> One useful technique is to crop the frame.

> The top half of the image is mostly sky. As humans, we know this isn't relevant to the curvature of the road, but the neural network is learning from scratch what patterns to look for in the image. It helps to remove parts of the image which aren't pertanant to driving.
>![](writeup_images/Masked2.png)

> ### Color Space
> I also tried converting the images into different color spaces. This can make certain parts of the image stand out. Of all the color spaces, I found LAB and HSV to be the most useful in creating accurate neural network models.
> #### LAB
>![](writeup_images/LAB.png)
> #### HSV
>![](writeup_images/HSV.png)

> ### Normalize Brightness
> Different parts of the road can be significantly brighter than others. This can create a false positive when the neural network is trying to identify the road markers. I dealt with this by normalizing the brightness of each image. 

> I achieved this by separating the HSV image into H,S,V channels. Then I normalized each image by dividing the image by np.max(V). This was an effective way of normalizing the brightness of all the images around the track

> ### Warping the Image
> I pulled a technique from the "Advanced Lane Finding" project. I warped the perspective of the image to be top-down. This was an effective way of feeding only the relevant information of the road into the neural network for training.
![](writeup_images/Warped.png)

> ### Mag Threshold
> Once the image was converted into a top-down perspective, I utilized another technique from the "Advanced Lane Finding" project. I applied a bilatteral filter, and then filtered the image by magnitude of the gradient.
![](writeup_images/final.png)

> ## Pipeline
> While I tried different techniques, the pipeline that gave the best results was to use the techniques from the "Advanced Lane Findnig" project. My final image processing pipeline was:
> * convert from BGR to RGB
> * Warp perspective
> * Bilateral Filter
> * Filter by magnitude of gradient

# Balancing the Data Set
> Driving around the track, most of the steering angles recorded will be straight. This can lead the neural network to conclude that the best approach is to drive straight all the time. To counteract this, I balanced the data set. That is, I duplicated some of the images which correspond to steering angles which are not zero so that the network gets more used to seeing those examples. This helps to not give the network a bias towards a zero steering angle.

> Another way of balancing the dataset was to give the model more examples of turning left and right. I did this by using the right and left camera images and then added a correction factor to the steering angle. Effectively, this simulated driving around the track slightly closer to the edge, and steering away from the edge.

> Finally, I also flipped each image and measurement in the data set. This had the effect of simulating driving around the track in the oposite direction.

> Below is a histogram of the data set before and after balancing the data. 
    <img src="./writeup_images/hist_1.png" width="400", height="200"/> 
    <img src="./writeup_images/hist_2.png" width="400", height="200"/>

# Neural Network
> After processing the images and balancing the data set, I passed the images and their measurements to the neural network for training.

> #### Over Fitting
> To avoid overfitting, I split the dataset into a training and validation set. In this project, the "test" set was testing the model driving around the track in autonomous mode. The validaton set was 20% of the final images.

> #### NN Model
> The model that I used is based on the [NVidia end to end model](https://images.nvidia.com/content/tegra/automotive/images/2016/solutions/pdf/end-to-end-dl-using-px.pdf). I removed a few of the layers for computational efficiency. I originally started with the LeNet architecture, but wasn't able to get successful results. I read up on some of the other students projects and comments in the slack channel and the project forums. More than one student had found success using this model, so I gave it a try and found it to be very robust.

> The final model in my project (model.py :: lines 227 - 245) consisted of :
* a lambda layer for normalizing the data.
* Three 5x5 convolutions
* Two 3x3 convolutions
* Three fully connected layers


# Final Output
> Once the model was trained, I used the drive.py script provided by Udacity to drive the car around the track in autonomous mode. [run1.mp4](run1.mp4) is a video of a successful autonomous lap around the track.

# Lessons Learned
> ### SANITY CHECK ALL YOUR DATA!
> In this project I learned the value of sanity checking all the data. Multiple times I would make a change to the preprocessing pipeline, but that change wasn't making it all the way to the neural network. For example, preprocessing images doesn't always happen "in place" so I had to create multiple arrays. Also, the model.py script reads images in through openCV which uses the BGR color space, but drive.py expects images in RGB color space. At other times, changes were being applied to either myData, or the UdacityData, but not both.

> I started building in checks that would check lenghts of arrays such that the script would sanity check that all the appropriate image processing steps had been taken. I also would show one of the images just before the data is passed to the neural network so that I can visually see that the process I intended to happen actually happened. This helped me multiple times when the images were in the wrong color space, or weren't what I expected them to be.

> #### Its all about the data
> One lesson I learned from reading through the forums and other students projects is that its all about the data. The quality and quantity of images going in has the greatest effect on the strength of the model. If the network hasn't seen enough examples of what to do in each situation, it can't predict the correct action or steering angle.

> #### Pickle the data
> Pickling the data allowed for much faster tests of changes to the pipeline. By pickling the data, I was able to read in the images and measurements at different parts of the pipeline much more quickly. If I made a change to the pipeline, I could read them in from scratch, but otherwise it saved a lot of time to pickle save and load the data.

# Possible Improvements
> A more robust pipeline would help the model to deal with more varied situations, such as the challenge track. If I were visiting this again, I would add in more controls to get rid of visual noise in the preprocessing steps. This could be done through color thresholds, or a more robust gradient filter.

> More training data would also help to create a more robust pipeline. Since I had computational limitations with my set up. I limited the data to only two laps around the track. That afforded me a pretty quick turn around on testing different ideas, but didn't create the most robust model.

> In reality, an actual car would need thousands of miles on various roads to really start to build a robust model for self driving.

# Acknowledgements
> I couldn't have done this project on my own. The udacity lessons were invaluable and much of the starting functionality was taken directly from the lessons, or from their video walk through of the project.

> I also read through some of the other students projects for ideas on what they had found usefull in areas such as the model architechture or color space they used. specifically, [Jeremy Shannon's](https://github.com/jeremy-shannon/CarND-Behavioral-Cloning-Project) pointed me in the direction of NVidia's neural network architechture.